In [1]:
import os
import pandas as pd 

In [2]:
file_path = '/Users/kevinshuey/massive_dataset/quant_intern/future_price20230628.txt'
df = pd.read_csv(file_path, sep='\t', header=0)

print(df.columns)

Index(['TDATE', 'TTIME', 'UPDATEMILLISEC', 'CONTRACTID', 'ExchangeInstID',
       'CONTRACTNAME', 'LASTPX', 'HIGHPX', 'LOWPX', 'CQ', 'CM', 'OC', 'TQ',
       'TM', 'LASTQTY', 'INITOPENINTS', 'OPENINTS', 'INTSCHG', 'RISELIMIT',
       'FALLLIMIT', 'PRESETTLE', 'PRECLOSE', 'S1', 'B1', 'SV1', 'BV1',
       'OPENPX', 'CLOSEPX', 'SETTLEMENTPX', 'LIFELOW', 'LIFEHIGH', 'AVGPX',
       'BIDIMPLYQTY', 'ASKIMPLYQTY', 'BSRATIO', 'SIDE', 'MFLXID', 'MFLXNAME',
       'S5', 'S4', 'S3', 'S2', 'B2', 'B3', 'B4', 'B5', 'SV5', 'SV4', 'SV3',
       'SV2', 'BV2', 'BV3', 'BV4', 'BV5', 'PREDELTA', 'CURRDELTA', 'LOCALTM',
       'MARKET', 'CHG', 'CHGPCT', 'VARIETIES', 'SETTLEGROUPID', 'SETTLEID',
       'UNIX', 'CLEARINGDAY', 'MFLAG', 'SOURCE', 'CONTRACTCODE', 'Offset',
       'TFLAG'],
      dtype='object')


In [3]:
df.head()

,TDATE,TTIME,UPDATEMILLISEC,CONTRACTID,ExchangeInstID,CONTRACTNAME,LASTPX,HIGHPX,LOWPX,CQ,...,VARIETIES,SETTLEGROUPID,SETTLEID,UNIX,CLEARINGDAY,MFLAG,SOURCE,CONTRACTCODE,Offset,TFLAG
0,20230628,85900,4,SI2308,NaN,NaN,13125.0,13125.0,13125.0,0,...,NaN,NaN,0,0,20230628,1.0,MDL,SI,32340004,1
1,20230628,90000,484,SI2308,NaN,NaN,13125.0,13125.0,13120.0,0,...,NaN,NaN,0,0,20230628,1.0,MDL,SI,32400484,1
2,20230628,90000,975,SI2308,NaN,NaN,13130.0,13130.0,13120.0,0,...,NaN,NaN,0,0,20230628,1.0,MDL,SI,32400975,1
3,20230628,90001,476,SI2308,NaN,NaN,13125.0,13130.0,13120.0,0,...,NaN,NaN,0,0,20230628,1.0,MDL,SI,32401476,1
4,20230628,90001,960,SI2308,NaN,NaN,13125.0,13130.0,13120.0,0,...,NaN,NaN,0,0,20230628,1.0,MDL,SI,32401960,1


数据以tick为单位传入，单位大约500ms。筛选数据中有用的字段：买一价`B1`, 卖一价`S1`, 成交总量`TQ`, 成交均价`AVGPX`。

可以根据`TQ`逐差算出每一个tick的成交量，以及每一个tick涉及的总金额。

给定买一价$x$ ，卖一价 $y$ ，成交均价 $z$ ，交易数量 $m$和交易总金额$T$；设买单数量为$b$, 卖单数量为$s$。有两个数量关系：

+ 总交易量：$b + s = m$
+ 成交均价：$b \cdot x +s \cdot y = T$

可以解得买单数$b = \frac{T - m \cdot y}{x - y}$，卖单数$s = m - b$。

In [5]:
df_u = df.loc[:,['TDATE', 'TTIME', 'UPDATEMILLISEC', 'B1', 'BV1', 'S1', 'SV1', 'TQ', 'TM', 'AVGPX', 'LASTPX']]
df_u.loc[:, 'quan'] = df_u['TQ'].diff().fillna(0)
df_u.head()

,TDATE,TTIME,UPDATEMILLISEC,B1,BV1,S1,SV1,TQ,TM,AVGPX,LASTPX,quan
0,20230628,85900,4,13125.0,12,13135.0,40,4537,297740625.0,13125.0,13125.0,0.0
1,20230628,90000,484,13120.0,27,13125.0,10,4551,298659350.0,13120.0,13125.0,14.0
2,20230628,90000,975,13125.0,5,13135.0,40,4583,300759775.0,13125.0,13130.0,32.0
3,20230628,90001,476,13125.0,5,13130.0,11,4592,301350600.0,13125.0,13125.0,9.0
4,20230628,90001,960,13120.0,3,13125.0,2,4601,301941225.0,13125.0,13125.0,9.0


In [ ]:
df_u['tot_amount'] = df_u['TQ'] * df_u['AVGPX']
df_u.loc[:, 'amount'] = df_u['tot_amount'].diff().fillna(0)
df_u['amount'] = df_u['quan'] * df_u['AVGPX']
df_u['ratio'] = df_u['tot_amount'] / df_u['TM']
df_u.head()

In [ ]:
df_u['buy_quan'] = (df_u['amount'] - df_u['quan'] * df_u['S1']) / (df_u['B1'] - df_u['S1'])
df_u['sell_quan'] = (df_u['amount'] - df_u['quan'] * df_u['B1']) / (df_u['S1'] - df_u['B1'])
df_u.head(20)
